# IST-01 (Kamitani lab) group seminar 2023

## Setup

In [ ]:
!pip install numpy
!pip install scikit-learn
!pip install matplotlib
!pip install seaborn
!pip install hdf5storage
!pip install bdpy

In [2]:
import os

import bdpy
import bdpy.ml
from bdpy.preproc import select_top
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns
import sklearn.svm
import sklearn.metrics

## Data downloading

Data are available at <https://figshare.com/s/d8cc32e991de0d6f56fe>.

In [ ]:
# Image-caption consistency
#fn = 'XXA230713_ImageSoundMatchCheck_volume_native_prep.h5'
#url  = 'https://figshare.com/ndownloader/files/41649960?private_link=d8cc32e991de0d6f56fe'

# Natural vs. AI-synthesized images
fn = 'XXB230713_NaturalArtificialImagePresentation_volume_native_prep.h5'
url  = 'https://figshare.com/ndownloader/files/41649513?private_link=d8cc32e991de0d6f56fe'

# Run the following command
!wget "$url" -O $fn
!ls

## Load data

In [ ]:
bdata = bdpy.BData(fn)
bdata.dataset.shape

## Data

- Target labels
- Run numbers
- Responses of the subject

In [ ]:
labels = bdata.get_labels('stimulus_name')
print(labels)

In [ ]:
labels_index = bdata.select('stimulus_name').flatten()
print(labels_index)

Stimulus list:

- [stimulus_list_ImageSoundMatchCheck.tsv](https://figshare.com/ndownloader/files/41649708?private_link=d8cc32e991de0d6f56fe)
- [stimulus_list_NaturalArtificialImagePresentation.tsv](https://figshare.com/ndownloader/files/41649711?private_link=d8cc32e991de0d6f56fe)

In [ ]:
runs = bdata.select('Run')
print(runs)

In [ ]:
response = bdata.select('response')
print(response)

## ROI

The distributed dataset contains ROIs created based on HCP 180 parcellation (HCP_MMP1.0 [Glasser et al., 2016](https://www.nature.com/articles/nature18933)). The HCP 180 parcellation defines 180 brain regions (per hemisphere) based on multi-modal MRI data (anatomy, task fMRI, functional connectivity, ...), covering the whole cortex.

![](https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fnature18933/MediaObjects/41586_2016_Article_BFnature18933_Fig3_HTML.jpg?as=webp)

From <https://www.nature.com/articles/nature18933/figures/3>

- [List of 180 parcells](https://gist.github.com/ShuntaroAoki/e47e901adfc36f167d4aeb15ed1ac66a)

| Index | Region | Abbreviation | ROIs |
| --- | --- | --- | --- |
| 1 | Primary visual cortex | PVC | V1 |
| 2 | Early visual cortex | EVC | V2, V3, V4 |
| 3 | Dorsal stream visual cortex | DSVC | V3A, V7, V3B, V6, V6A, IPS1 |
| 4 | Ventral stream visual cortex | VSVC | V8, VVC, VMV1, VMV2, VMV3, PIT, FFC |
| 5 | MT+ complex and neighboring visual areas | MTcVA | V3CD, LO1, LO2, LO3, MT, MST, V4t, FST, PH |
| 6 | Somatosensory and motor cortex | SMC | 4, 3a, 3b, 1, 2 |
| 7 | Paracentral lobular and mid cingulate cortex | PCL_MCC | 5L, 5m, 5mv, 24dd, 24dv, 6mp, 6ma, SCEF |
| 8 | Premotor cortex | PMC | 6a, 6d, FEF, PEF, 55b, 6v, 6r |
| 9 | Posterior opercular cortex | POC | 43, FOP1, OP4, OP2-3, OP1, PFcm |
| 10 | Early auditory cortex | EAC | A1, MBelt, LBelt, PBelt, RI |
| 11 | Auditory association cortex | AAC | A4, A5, STSdp, STSda, STSvp, STSva, TA2, STGa |
| 12 | Insular and frontal opercular cortex | IFOC | 52, PI, Ig, PoI1, PoI2, FOP2, Pir, AAIC, MI, FOP3, FOP4, FOP5, AVI |
| 13 | Medial temporal cortex | MTC | H, PreS, EC, PeEc, PHA1, PHA2, PHA3 |
| 14 | Lateral temporal cortex | LTC | TGd, TGv, TF, TE2a, TE2p, TE1a, TE1m, TE1p, PHT |
| 15 | Temporo-parieto-occipital junction | TPOJ | TPOJ2, TPOJ3, TPOJ1, STV, PSL |
| 16 | Superior parietal cortex | SPC | MIP, LIPv, VIP, LIPd, AIP, 7PC, 7Am, 7AL, 7Pm, 7PL |
| 17 | Inferior parietal cortex | IPC | PGp, IP0, IP1, IP2, PF, PFt, PFop, PFm, PGi, PGs |
| 18 | Posterior cingulate cortex | PCC | DVT, ProS, POS2, POS1, RSC, 7m, PCV, v23ab, d23ab, 31pv, 31pd, 31a, 23c,
  23d |
| 19 | Anterior cingulate and medial prefrontal  cortex | ACC_mPFC | 33pr, a24pr, p24pr, p24, a24, p32pr, a32pr, d32, p32, s32, 8BM, 9m, 10r,
  10v, 25 |
| 20 | Orbital and polar frontal cortex | OPFC | OFC, pOFC, 13l, 11l, 47s, 47m, a47r, 10pp, a10p, p10p, 10d |
| 21 | Inferior frontal cortex | IFC | 44, 45, 47l, IFJp, IFJa, IFSp, IFSa, p47r |
| 22 | Dorsolateral prefrontal cortex | DLPFC | SFL, s6-8, i6-8, 8BL, 8Ad, 8Av, 8C, 9p, 9a, 9-46d, a9-46v, p9-46v, 46 |

In [ ]:
bdata.show_metadata()

In [ ]:
roi = 'V1'
roi_select = f'"hcp180_r_lh.L_{roi}_ROI" + "hcp180_r_rh.R_{roi}_ROI"'

x = bdata.select(roi_select)
x.shape

In [ ]:
# Early auditory cortex
rois = ['A1', 'MBelt', 'LBelt', 'PBelt', 'RI']
roi_select = ' + '.join([f'"hcp180_r_lh.L_{roi}_ROI" + "hcp180_r_rh.R_{roi}_ROI"' for roi in rois])

x = bdata.select(roi_select)
x.shape